In [17]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import pandas as pd
import random
import csv
import os
import math
import numpy as np
from tqdm import tqdm
from numba import jit

In [2]:
traindf = pd.read_pickle("./pickles/df_small_train.pkl")
testdf = pd.read_pickle("./pickles/df_small_test.pkl")

# run if you want to use the whole train/test dataset
# traindf = pd.read_csv("data/training_set_VU_DM.csv")
# traindf = traindf[:300]

In [3]:
def competitors(df):
    """
    Make a new column in the dataframe (competitor_bool) for when there 
    exists a competitor, 1 is True, 0 is False.
    """
    competitor_bools = []
    compare_cols = ["comp1_rate", "comp2_rate", "comp3_rate", "comp4_rate", "comp5_rate",
                "comp6_rate", "comp7_rate", "comp8_rate"]

    for index, row in df.iterrows():
        comp_bool = 0

        # compare_cols_total exists of a column of data from comp_rate and 
        # comp_inv. So, a combination of competitor price and room-availability. 
        for competitor in compare_cols:
            if row[competitor] == 1:
                comp_bool = 1
        competitor_bools.append(comp_bool)
    
    # Append list of bools to new column
    df["competitor_bool"] = competitor_bools
    
    return df

In [4]:
def visitor_history(df):
    """
    Add column that tells us whether someone has visited a hotel before.
    Column name = total_visited; 1 is True, 0 is False.
    """
    
    # Get none-missing values
    hist_starrating = df.visitor_hist_starrating.isna()
    hist_adr = df.visitor_hist_adr_usd.isna()

    # Dit kan waarschijnlijk veel mooier en sneller maar het werkt...
    total_visited = []
    for index, row in df.iterrows():
        if hist_starrating[index] or hist_adr[index]:
            visited = 0
        else:
            visited = 1
        total_visited.append(visited)

    df["total_visited"] = total_visited
    
    return df

In [5]:
def add_score(df):
    """
    Add a score 
    """
    if df["booking_bool"] == 1 or df["click_bool"] == 1:
        score = 1
    else:
        score = 0
    return score


def preprocessing(traindf):
    traindf = competitors(traindf)
    traindf = visitor_history(traindf)
    
    # Without price_quality because of floating error (TODO)
    df = traindf[["prop_id", "srch_id", "position", "competitor_bool", "total_visited", "click_bool", "booking_bool"]]
    
    df['score'] = df.apply(add_score , axis=1)
    
    return df

In [6]:
df = preprocessing(traindf)
df.head(10)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,prop_id,srch_id,position,competitor_bool,total_visited,click_bool,booking_bool,score
1196703,2990,79994,24,0,0,0,0,0
1196704,4256,79994,6,0,0,1,1,1
1196705,6427,79994,17,0,0,0,0,0
1196706,8289,79994,30,0,0,0,0,0
1196707,13668,79994,21,0,0,0,0,0
1196708,17105,79994,28,1,0,0,0,0
1196709,19743,79994,12,0,0,0,0,0
1196710,21688,79994,7,0,0,0,0,0
1196711,28818,79994,19,0,0,0,0,0
1196712,37093,79994,27,1,0,0,0,0


In [7]:
y = df["prop_id"]
X = df.copy()
# X = df.drop("prop_id", axis=1).copy()

# TODO: test en trainingset maken
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [8]:
# prediction = rfr.predict(X_test)
# prediction_proba = rfr.predict_proba(X_test)
# print(r2_score(prediction, y_test))

In [9]:
def dcg_from_ranking(y_true, ranking):
    """Discounted cumulative gain (DCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    ranking : array-like, shape = [k]
        Document indices, i.e.,
            ranking[0] is the index of top-ranked document,
            ranking[1] is the index of second-ranked document,
            ...
    k : int
        Rank.
    Returns
    -------
    DCG @k : float
    """
    y_true = np.asarray(y_true)
    ranking = np.asarray(ranking)
    rel = y_true[ranking]
    gains = 2 ** rel - 1
    discounts = np.log2(np.arange(len(ranking)) + 2)
    return np.sum(gains / discounts)


def ndcg_from_ranking(y_true, ranking):
    """Normalized discounted cumulative gain (NDCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    ranking : array-like, shape = [k]
        Document indices, i.e.,
            ranking[0] is the index of top-ranked document,
            ranking[1] is the index of second-ranked document,
            ...
    k : int
        Rank.
    Returns
    -------
    NDCG @k : float
    """
    k = len(ranking)
    best_ranking = np.argsort(y_true)[::-1]
    best = dcg_from_ranking(y_true, best_ranking[:k])
    return dcg_from_ranking(y_true, ranking) / best

def dcg_score(y_true, y_score, k=20):
    """Discounted cumulative gain (DCG) at rank K.
    """
    
    order = np.argsort(y_score)[::-1]
    k = len(y_true)
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


In [10]:
rank_pred_groups   = X.groupby('srch_id').agg({'position':lambda x: list(x)})

# Get unique srchids to group by
unique_srchid = X.srch_id.unique()
rfc = RandomForestClassifier(n_jobs=1)

In [30]:
# Oh jit kan niet met pandas Series werken, moet omgeschreven worden naar np arrays
# @jit(nopython=True)

def calc_score_predictions():    
    scores = []
    
    # Iterate over every group of search_ids
    for index in range(1, len(unique_srchid)):

        # Filter slices on unique srch_id
        slicer = X.loc[X['srch_id'] == unique_srchid[index]]
        y = slicer.prop_id

        # Fit seperate model; make prediction for every slice
        model = rfc.fit(slicer, y)
        predictions = model.predict_proba(slicer)[:, 1]

        y_true = list(predictions)
        rank = list(slicer.position)

        # Compare predictions with actual position
        best = dcg_score(y_true, y_true)
        actual  = dcg_score(y_true, rank)
        scores.append(actual / best)

        return np.mean(scores), predictions, y


score, last_prediction, last_position  = calc_score_predictions()
print(last_prediction)
print(last_position)
print("score: ", score)

[0.2 0.9 0.1 0.  0.  0.3 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
1050185      2312
1050186      2369
1050187      7071
1050188     10681
1050189     13398
1050190     15439
1050191     21231
1050192     27221
1050193     42127
1050194     42723
1050195     45667
1050196     48556
1050197     48900
1050198     51617
1050199     52204
1050200     52283
1050201     58641
1050202     73138
1050203     73474
1050204     80863
1050205    101135
1050206    102345
1050207    107698
1050208    111905
1050209    113504
1050210    119669
1050211    121462
1050212    121896
1050213    124538
1050214    134598
1050215    138555
1050216    140396
Name: prop_id, dtype: int64
score:  0.2875952954295652


In [14]:
propid_pred_groups = X.groupby('srch_id').agg({'prop_id':lambda x: list(x)})
rank_pred_groups   = X.groupby('srch_id').agg({'position':lambda x: list(x)})

logger = lambda x: math.log(x + 1, 2)
X['log_rank'] = X.groupby(by = 'srch_id')['position'].rank(ascending = False).map(logger)

# sum( (2 ** points - 1) / log2(rank_in_results + 1) )

In [26]:
propid_pred_groups.head(10)

# scores = []
# for search_id in range(1, len(propid_pred_groups)):
#     y_true = list(propid_pred_groups[:search_id]["prediction_prop_id"])[0]
#     rank = list(rank_pred_groups[:search_id]["position"])[0]
#     print(y_true, rank)
#     score = ndcg_from_ranking(y_true, rank)
#     scores.append(score)

# print(scores)

,prop_id
srch_id,
124,"[3709, 11719, 12082, 13297, 15174, 17345, 2146..."
218,"[19663, 32400, 73307, 108073, 132936]"
439,"[9095, 9826, 12860, 16708, 18660, 20110, 23639..."
597,"[1988, 10087, 95874, 97465, 102644, 135952]"
870,"[12720, 21234, 23504, 31806, 35718, 42391, 424..."
1237,"[11396, 14133, 22765, 23121, 37255, 46859, 543..."
1350,"[4375, 8469, 17709, 20751, 35653, 43705, 44794..."
1384,"[1172, 3054, 11243, 13109, 22967, 31977, 36891..."
1545,"[16371, 16699, 18906, 19070, 19804, 27448, 290..."


In [27]:
rank_pred_groups.head(10)

,position
srch_id,
124,"[33, 35, 36, 2, 10, 30, 3, 29, 12, 26, 15, 20,..."
218,"[1, 6, 4, 3, 2]"
439,"[26, 24, 25, 21, 27, 3, 12, 14, 6, 13, 1, 28, ..."
597,"[2, 6, 7, 1, 4, 3]"
870,"[24, 9, 14, 18, 27, 7, 29, 22, 15, 19, 8, 1, 2..."
1237,"[4, 13, 26, 14, 21, 15, 10, 16, 1, 24, 19, 27,..."
1350,"[13, 9, 21, 19, 14, 6, 26, 7, 12, 30, 22, 24, ..."
1384,"[29, 6, 9, 10, 24, 8, 13, 19, 7, 18, 4, 25, 1,..."
1545,"[33, 9, 6, 13, 30, 31, 4, 8, 24, 20, 1, 27, 15..."
